In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [2]:
from client import Client
from server_algo import Server_with_Algorithm
from server import Server
from datasets import load_dataset, concatenate_datasets
import validation

In [3]:
# 1.首先选择要进行实验的client
clients = [Client(vectorstore_path="common_sense_db_0.6b")]
            # Client(vectorstore_path="computer_science_coding_related_db_0.6b"),
            # Client(vectorstore_path="law_related_db_0.6b"), Client(vectorstore_path="medicine_related_db_0.6b")]
# 2.对每个client进行加载
for c in clients:
    c.load_vectorstore()

/home/weiwei/RAGnet/client.py:63: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_path,


Vectorstore common_sense_db_0.6b loaded.


In [4]:
# 3.创建server对象，这里需要选择模型，请预先在ollama上部署
# server = Server(model_name="qwen3:4b")
server = Server(model_name="qwen3:4b")

/home/weiwei/RAGnet/server.py:21: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  self.llm = ChatOllama(model=model_name)


In [5]:
# 4.加载测试数据集，这里以trivia_qa为例子，这里取前100个
from datasets import Dataset, DatasetDict, load_dataset
from pathlib import Path

root_dir = "./test_dataset/trivia_qa_test"
parquet_files = []

# 递归查找所有符合的文件
for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        parquet_files.append(os.path.join(dirpath, filename))

print(f"找到 {len(parquet_files)} 个 validation parquet 文件")

# 加载所有文件为一个dataset列表
datasets_list = load_dataset("arrow", data_files={"train": parquet_files}, split="train")

找到 3 个 validation parquet 文件


In [6]:
# 抽样选取 100 条（打乱顺序）
random_samples = datasets_list.shuffle(seed=42).select(range(100))

In [ ]:
# 5.进行评估，并生成csv文件
validation.evaluate_datasets(clients, server, samples=random_samples, top_k=10,
                        output_csv="trivia_qa_results_baseline_topk_10.csv", dataset_name="trivia_qa")

Processing question: What is the US state capital of Florida?
Gold answers: ['Tallahasee', 'Talahassee', 'Tallahassee', 'Tallahassee, Fl', 'Tallahassee, FL.', 'Tallahassee (FL)', 'Swift Creek Middle School', 'Flag of Tallahassee, FL', 'Tallahasse', 'Tallahassee, Florida', 'Tallahassee, FL', 'Tallahassee, Fl.', 'Capital of Florida', 'UN/LOCODE:USTLH']
uncleaned answer: <think>
Okay, let's tackle this question: "What is the US state capital of Florida?" 

First, I need to look through the provided contexts and metadata to find the answer. The user has given a list of contexts, so I'll go through them one by one.

Starting with Context 1: It mentions "List of places in California (Z)" but that's about California, not Florida. So probably not relevant.

Context 2 is about Jacksonville, disambiguation. It says Jacksonville is the largest city in Florida. But the question is about the capital, not the largest city. However, maybe there's more info here. The context lists Jacksonville as a ci